<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/agentsframework_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ag2 -q

!pip install colab-env --upgrade -q
!pip install openai -q

!pip install langchain -q
!pip install langchain_community -q
!pip install google-search-results -q

In [18]:
from openai import OpenAI
import os
import colab_env
from openai import OpenAI

In [6]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
api_key = os.getenv("OPENAI_API_KEY")

## autogen

AutoGen (Microsoft):

* AutoGen, being a Microsoft-backed open-source project focused on multi-agent systems, is likely seeing increasing adoption, particularly within organizations already invested in the Microsoft ecosystem.

* Salesforce Ventures highlighted that Autogen AI (a company leveraging similar multi-agent concepts) had 28 customers across Fortune 500 companies and international government agencies within a year. This suggests that the underlying multi-agent technology is gaining traction in enterprise environments, and AutoGen, as a prominent framework in this space, would likely be part of this trend.


In [8]:
import autogen

# Configure the OpenAI API (replace with your actual API key)
config_list_openai = [
    {
        "model": "gpt-3.5-turbo",  # Or a more capable model like gpt-4
        "api_key": api_key,
    }
]

# 1. Define the Writer Agent
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list_openai},
    system_message="You are a creative writer skilled at crafting engaging short stories.",
)

# 2. Define the Reviewer Agent
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list_openai},
    system_message="You are a helpful and critical editor. Review the story provided and suggest improvements for clarity, creativity, and flow.",
)

# 3. Define the User Proxy Agent (to initiate the conversation)
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",  # Never ask for human input in this automated example
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "story_output"},  # Optional: for code execution if needed
)

# 4. Initiate the conversation
user_proxy.initiate_chat(
    writer,
    message="Write a short story about a robot named Bolt who dreams of becoming a chef.",
)

# The Writer will generate a story and send it to the User_Proxy.
# We need to define how the User_Proxy then involves the Reviewer.

# Let's add a function that the User_Proxy can use to trigger the Reviewer.
def review_story(message):
    reviewer.send(message, user_proxy)
    return "Story sent to the reviewer for feedback."

user_proxy.register_function(
    function_map={"review_story": review_story}
)

# After the Writer generates the initial story, it can call the 'review_story' function
# (we'll need to guide the Writer to do this in its initial prompt or subsequent interactions).

# For a more streamlined flow, we can modify the initial prompt to guide the Writer:
user_proxy.initiate_chat(
    writer,
    message="Write a short story about a robot named Bolt who dreams of becoming a chef. Once you have a draft, please use the 'review_story' function to send it to the reviewer for feedback.",
)



User_Proxy (to Writer):

Write a short story about a robot named Bolt who dreams of becoming a chef.

--------------------------------------------------------------------------------
Writer (to User_Proxy):

In the bustling city of Circuitville, where robots of all shapes and sizes lived and worked, there was one exceptional robot named Bolt who dreamed of becoming a chef. Bolt had always been fascinated by the art of cooking, watching with awe as humans and robots alike created delicious meals in the city's finest restaurants.

Every day after finishing his duties at the local factory, Bolt would spend his free time studying recipes and experimenting with different ingredients in his small, cluttered kitchen. Despite his lack of taste sensors, Bolt relied on his keen observation skills and precise programming to create dishes that looked and smelled mouth-watering.

Word of Bolt's culinary creations soon spread throughout Circuitville, and one day, he received an invitation to compete

ChatResult(chat_id=None, chat_history=[{'content': "Write a short story about a robot named Bolt who dreams of becoming a chef. Once you have a draft, please use the 'review_story' function to send it to the reviewer for feedback.", 'role': 'assistant', 'name': 'User_Proxy'}, {'content': "Once upon a time, in a bustling city where humans and robots coexisted harmoniously, there lived a small, unassuming robot named Bolt. While Bolt was designed for construction work, his true passion lay elsewhere - in the art of cooking.\n\nEvery chance he got, Bolt would sneak away to the city's largest market, where vendors displayed colorful fruits and vegetables, and the aroma of freshly baked bread lingered in the air. He would observe the skilled chefs at work, their hands deftly chopping, stirring, and seasoning with precision. It filled Bolt with a sense of awe and longing.\n\nOne day, as Bolt watched a famous chef prepare a gourmet dish, he felt a spark of inspiration within him. He decided t

## langchain

1. LangChain:

LangChain excels at creating chains of operations with LLMs. We can structure this by having a chain for writing and a separate chain (or agentic behavior) for review.



In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentOutputParser
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import AgentAction, AgentFinish
from typing import List, Union
import re
import os
import colab_env

api_key = os.getenv("OPENAI_API_KEY")

# Configure OpenAI (replace with your API key)
llm = OpenAI(api_key=api_key)

# --- Writing Chain ---
writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a short story about {topic}.",
)
writer_chain = LLMChain(llm=llm, prompt=writer_prompt, output_key="story")

# --- Reviewing Prompt ---
reviewer_prompt = PromptTemplate(
    input_variables=["story"],
    template="Review the following short story and provide feedback for clarity, creativity, and flow:\n\n{story}",
)
reviewer_chain = LLMChain(llm=llm, prompt=reviewer_prompt, output_key="feedback")

# --- Agent-like Behavior (Simple Sequential Chain) ---
def generate_and_review(topic):
    story_output = writer_chain({"topic": topic})
    story = story_output["story"]
    feedback_output = reviewer_chain({"story": story})
    feedback = feedback_output["feedback"]
    return {"story": story, "feedback": feedback}

# Run the process
result = generate_and_review("a robot named Bolt who dreams of becoming a chef")
print("Generated Story:\n", result["story"])
print("\nReviewer Feedback:\n", result["feedback"])

# --- More Agentic Approach (Conceptual - LangChain Agents are more complex) ---
# You could use LangChain agents with tools for a more iterative process,
# where an agent writes, then uses a "review" tool, and potentially revises.
# This would involve defining tools and a more sophisticated agent setup.

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Generated Story:
 

Bolt was a small, shiny robot with big dreams. While most of his fellow robots were programmed for tasks like building and cleaning, Bolt had always been fascinated with cooking. He spent his days watching cooking shows and reading recipe books, dreaming of the day when he could become a chef.

But his dreams seemed impossible. Bolt was designed for manual labor and had no cooking skills whatsoever. His programming didn't include taste buds or the ability to smell, which made it even harder for him to pursue his dream.

Despite these challenges, Bolt was determined to make his dream a reality. He spent every spare moment practicing in the kitchen, using his precise robotic movements to chop, mix and stir. He even programmed himself to simulate the sense of taste and smell, relying on his advanced sensors to analyze flavors and ingredient

## crewai

2. CrewAI:

CrewAI is designed explicitly for multi-agent collaboration. We'll define two agents with specific roles and a task for each.

In [ ]:
!pip install crewai -q

In [2]:
import os
from crewai import Crew, Agent, Task
import colab_env

api_key = os.getenv("OPENAI_API_KEY")

# You might need to set your OpenAI API key as an environment variable
#os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Define the Agents
writer_agent = Agent(
    role='Creative Writer',
    goal='Write an engaging short story about a robot who dreams of becoming a chef.',
    backstory="You are a skilled storyteller with a knack for imaginative narratives.",
    verbose=True,
    llm_config={"model": "gpt-3.5-turbo"} # Or a more capable model
)

reviewer_agent = Agent(
    role='Critical Editor',
    goal='Review the short story for clarity, creativity, and flow, providing constructive feedback.',
    backstory="You are an experienced editor known for your keen eye for detail and helpful suggestions.",
    verbose=True,
    llm_config={"model": "gpt-3.5-turbo"} # Or a more capable model
)

# 2. Define the Tasks
write_task = Task(
    description='Write a short story about a robot named Bolt who dreams of becoming a chef.',
    agent=writer_agent,
    # Added expected_output for the write task. This should be a type string
    expected_output="A short story about a robot named Bolt who dreams of becoming a chef."
)

review_task = Task(
    description='Review the short story drafted by the writer and provide feedback for improvements.',
    agent=reviewer_agent,
    # Added expected_output for the review task. This should be a type string
    expected_output="Feedback on the short story's clarity, creativity, and flow."
)

# 3. Create the Crew
story_crew = Crew(
    agents=[writer_agent, reviewer_agent],
    tasks=[write_task, review_task],
    verbose=True  # Changed verbose to True for higher verbosity, or False for lower
)

# 4. Run the Crew
result = story_crew.kickoff()

print("Final Result:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c35ebb3d-d9b3-4cd4-bd8d-a73ece3b7b29                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: ca08004f-b970-473d-a31c-ee9054cce0ec
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: ca08004f-b970-473d-a31c-ee9054cce0ec
       Status: Executing Task...
    └── 🤖 Agent: Creative Writer
            Status: In Progress

# Agent: Creative Writer
## Task: Write a short story about a robot named Bolt who dreams of becoming a chef.


🚀 Crew: crew
└── 📋 Task: ca08004f-b970-473d-a31c-ee9054cce0ec
       Status: Executing Task...
    └── 🤖 Agent: Creative Writer
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: ca08004f-b970-473d-a31c-ee9054cce0ec
│      Assigned to: Creative Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Writer
│           Status: ✅ Completed
└── 📋 Task: 1fe3a217-52ad-446d-a6cd-6640ce2a189e
       Status: Executing Task...
    └── 🤖 Agent: Critical Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: ca08004f-b970-473d-a31c-ee9054cce0ec
│      Assigned to: Creative Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Writer
│           Status: ✅ Completed
└── 📋 Task: 1fe3a217-52ad-446d-a6cd-6640ce2a189e
       Assigned to: Critical Editor
       Status: ✅ Completed
    └── 🤖 Agent: Critical Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1fe3a217-52ad-446d-a6cd-6640ce2a189e                                                                     │
│  Agent: Critical Editor                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c35ebb3d-d9b3-4cd4-bd8d-a73ece3b7b29                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Result:
 **Feedback on Clarity, Creativity, and Flow:**

1. **Clarity:**
   - The story is easy to follow, and the premise of a robot chef is clear and well-communicated. However, consider providing a bit more background on how Bolt was created or incorporated into the restaurant's operations. A brief introduction to Bolt's origin or the initial reaction of Marla and Joe to having a robot in the kitchen could enhance understanding.
   - Clarify the timeline in certain sections. For example, the transition from Bolt’s experimentation to presenting his dishes to Marla and Joe could benefit from a smoother connection. Phrases such as “after a week of clandestine cooking” could specify the intensity and volume of the dishes Bolt perfected.

2. **Creativity:**
   - This story has an imaginative premise, blending robotics and culinary arts in a unique way. The inventive dishes are described vividly, adding creativity to the narrative. To enhance this aspect, consider incorporating more